A [typical workflow](https://medium.com/planet-stories/a-gentle-introduction-to-gdal-part-4-working-with-satellite-data-d3835b5e2971) to create an image from raw satellite data would be:

* Download data.
* Re-order or assemble bands into the desired order (red, green, blue; or near-infrared, red, green; etc.)
* Increase the resolution with pan-sharpening, if desired.
* Contrast-stretch and color-correct the imagery, either algorithmically or by hand.
* Restore georeferencing information, if necessary.
* Crop, re-project, and re-size image to merge with other data.

GDAL - [Geospatial Data Abstraction Library](https://www.gdal.org/gdal_tutorial.html)

# Getting the Data


[The wildfire in California on Nov. 2018](https://github.com/DataForGood-Norway/FireFromSpace#spotted-fires-from-space)

The steps needed to download the imagery data:

1. Locate on geojson.io to get the coordinates: http://geojson.io/#map=13/36.2200/-118.6188
1. Define an Area of Interest (AOI): AOI is the location/geographical window out of which we want to get data.

![](./images/geojson_io.jpeg)

You get this output

```json
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -118.71156692504884,
              36.161716102717776
            ],
            [
              -118.54196548461914,
              36.161716102717776
            ],
            [
              -118.54196548461914,
              36.27970720524017
            ],
            [
              -118.71156692504884,
              36.27970720524017
            ],
            [
              -118.71156692504884,
              36.161716102717776
            ]
          ]
        ]
      }
    }
  ]
}
```

In [1]:
from geojson import Feature, Point, FeatureCollection
from pprint import pprint


feature_collection = FeatureCollection({
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -118.71156692504884,
              36.161716102717776
            ],
            [
              -118.54196548461914,
              36.161716102717776
            ],
            [
              -118.54196548461914,
              36.27970720524017
            ],
            [
              -118.71156692504884,
              36.27970720524017
            ],
            [
              -118.71156692504884,
              36.161716102717776
            ]
          ]
        ]
      }
    }
  ]
})

### Save the AOI’s coordinates generated in GeoJSON format in Jupyter notebook

In [2]:
feature_collection["features"][0]["geometry"]["coordinates"]

[[[-118.71156692504884, 36.161716102717776],
  [-118.54196548461914, 36.161716102717776],
  [-118.54196548461914, 36.27970720524017],
  [-118.71156692504884, 36.27970720524017],
  [-118.71156692504884, 36.161716102717776]]]

In [3]:
# AOI co-ordinates (created via geojson.io) 
geojson_geometry = feature_collection["features"][0]["geometry"]
pprint(geojson_geometry)

{'coordinates': [[[-118.71156692504884, 36.161716102717776],
                  [-118.54196548461914, 36.161716102717776],
                  [-118.54196548461914, 36.27970720524017],
                  [-118.71156692504884, 36.27970720524017],
                  [-118.71156692504884, 36.161716102717776]]],
 'type': 'Polygon'}


###  Create filters for the date range, cloud coverage, and geometry.
This will enable us to further constrain our Data API search.

In [4]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2018-10-01T00:00:00.000Z",
    "lte": "2018-12-30T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

### Planet API Key


To use Planet’s APIs, you’ll need an API key.
Create an account(14-day trial) at Planet Explorer and access the API key from here.

In [5]:
PLANET_API_KEY = "6979652b63944b07bc6eac35d669b3e3"

In [6]:
# See requirements.txt to set up your dev environment.
import sys
import os
import json
import scipy
import urllib
import datetime 
import urllib3
import rasterio
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
from osgeo import gdal
from planet import api
from planet.api import filters
from traitlets import link
import rasterio.mask as rio_mask
from shapely.geometry import mapping, shape
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
urllib3.disable_warnings()
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

%matplotlib inline
# will pick up api_key via environment variable PL_API_KEY
# but can be specified using `api_key` named argument
# api_keys = json.load(open("apikeys.json",'r'))
# client = api.ClientV1(api_key=api_keys["PLANET_API_KEY"])
client = api.ClientV1(api_key=PLANET_API_KEY)

In [13]:
# Basemap Mosaic (v1 API)
mosaicsSeries = 'global_quarterly_2017q1_mosaic'
# Planet tile server base URL (Planet Explorer Mosaics Tiles)
mosaicsTilesURL_base = 'https://tiles0.planet.com/experimental/mosaics/planet-tiles/' + mosaicsSeries + '/gmap/{z}/{x}/{y}.png'
# Planet tile server url
mosaicsTilesURL = mosaicsTilesURL_base + '?api_key=' + PLANET_API_KEY
# Map Settings 
# Define colors
colors = {'blue': "#009da5"}
# Define initial map center lat/long
center = [36.2200414612719, -118.6187839508056] # 36.22004146127195&lng=-118.6187839508056
# Define initial map zoom level
zoom = 11
# Set Map Tiles URL
planetMapTiles = TileLayer(url= mosaicsTilesURL)
# Create the map
m = Map(
    center=center, 
    zoom=zoom,
    default_tiles = planetMapTiles # Uncomment to use Planet.com basemap
)
# Define the draw tool type options
polygon = {'shapeOptions': {'color': colors['blue']}}
rectangle = {'shapeOptions': {'color': colors['blue']}} 

# Create the draw controls
# @see https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L293
dc = DrawControl(
    polygon = polygon,
    rectangle = rectangle
)
# Initialize an action counter variable
actionCount = 0
AOIs = {}

# Register the draw controls handler
def handle_draw(self, action, geo_json):
    # Increment the action counter
    global actionCount
    actionCount += 1
    # Remove the `style` property from the GeoJSON
    geo_json['properties'] = {}
    # Convert geo_json output to a string and prettify (indent & replace ' with ")
    geojsonStr = json.dumps(geo_json, indent=2).replace("'", '"')
    AOIs[actionCount] = json.loads(geojsonStr)
    
# Attach the draw handler to the draw controls `on_draw` event
dc.on_draw(handle_draw)
m.add_control(dc)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [14]:

myAOI = geojson_geometry

# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes

start = datetime.datetime(year=2018,month=10,day=18)
stop = datetime.datetime(year=2018,month=12,day=18)

query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('cloud_cover', lt=50),
    filters.date_range('acquired', gt=start, lt=stop)
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band'] 
) # PSScene3Band

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
scenes = []
planet_map = {}
for item in result.items_iter(limit=500):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item['id']
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    scenes.append(props)
scenes = pd.DataFrame(data=scenes)
display(scenes)
print(len(scenes))

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,quality_category,rows,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle
0,2018-12-17T18:12:36.621055Z,0.00,0.88,8977,32611,"{'coordinates': [[[-118.6946908234269, 36.1363...",True,3.9,20181217_181236_1005,PS2,...,standard,4637,1005,1940776,154.9,26.3,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:05:23.956Z,0,0.1
1,2018-12-17T18:12:34.552214Z,0.00,0.45,8956,32611,"{'coordinates': [[[-118.65239571447013, 36.294...",True,3.8,20181217_181234_1005,PS2,...,standard,4631,1005,1940776,155.0,26.1,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:04:45.006Z,0,0.1
2,2018-12-17T18:14:22.436537Z,0.00,0.92,8894,32611,"{'coordinates': [[[-118.64915467527891, 36.077...",False,3.8,20181217_181422_103e,PS2,...,test,4622,103e,1940782,154.9,26.2,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:26:30.431Z,0,2.5
3,2018-12-17T18:14:21.402641Z,0.00,0.94,8874,32611,"{'coordinates': [[[-118.88762454600982, 36.264...",False,3.8,20181217_181421_103e,PS2,...,test,4622,103e,1940782,154.9,26.1,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:26:51.214Z,0,2.5
4,2018-12-17T18:14:20.368745Z,0.00,0.90,8861,32611,"{'coordinates': [[[-118.86996154384987, 36.330...",True,3.8,20181217_181420_103e,PS2,...,standard,4626,103e,1940782,155.0,26.1,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:26:44.188Z,0,2.5
5,2018-12-17T18:12:35.586634Z,0.00,0.71,8961,32611,"{'coordinates': [[[-118.66854499366889, 36.234...",True,3.9,20181217_181235_1005,PS2,...,standard,4639,1005,1940776,155.0,26.2,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:05:12.106Z,0,0.1
6,2018-12-17T18:14:19.334849Z,0.00,0.76,8868,32611,"{'coordinates': [[[-118.85103662476965, 36.396...",True,3.8,20181217_181419_103e,PS2,...,standard,4619,103e,1940782,155.0,26.0,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18T04:26:46.723Z,0,2.5
7,2018-12-16T17:45:17.548397Z,0.02,0.01,8282,32611,"{'coordinates': [[[-118.5883077608885, 36.2543...",True,3.6,20181216_174517_1052,PS2,...,standard,3993,1052,1938040,148.6,23.5,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-17T04:14:01.825Z,0,0.1
8,2018-12-16T17:45:16.593668Z,0.00,0.00,8300,32611,"{'coordinates': [[[-118.85039421411327, 36.218...",True,3.6,20181216_174516_1052,PS2,...,standard,3993,1052,1938040,148.6,23.6,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-17T04:14:18.252Z,0,0.1
9,2018-12-16T17:45:15.63894Z,0.00,0.02,8272,32611,"{'coordinates': [[[-118.83447785054126, 36.157...",True,3.6,20181216_174515_1052,PS2,...,standard,3994,1052,1938040,148.6,23.6,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-17T04:14:01.868Z,0,0.1


331


In [15]:
scenes.shape

(331, 26)

In [16]:
# now let's clean up the datetime stuff
# make a shapely shape from our aoi
portland = shape(myAOI)
footprints = []
overlaps = []
# go through the geometry from our api call, convert to a shape and calculate overlap area.
# also save the shape for safe keeping
for footprint in scenes["geometry"].tolist():
    s = shape(footprint)
    footprints.append(s)
    overlap = 100.0*(portland.intersection(s).area / portland.area)
    overlaps.append(overlap)
# take our lists and add them back to our dataframe
scenes['overlap'] = pd.Series(overlaps, index=scenes.index)
scenes['footprint'] = pd.Series(footprints, index=scenes.index)
# now make sure pandas knows about our date/time columns.
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])
scenes.head()

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
0,2018-12-17 18:12:36.621055+00:00,0.0,0.88,8977,32611,"{'coordinates': [[[-118.6946908234269, 36.1363...",True,3.9,20181217_181236_1005,PS2,...,1005,1940776,154.9,26.3,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18 04:05:23.956000+00:00,0,0.1,13.582701,POLYGON ((-118.6946908234269 36.13635564648786...
1,2018-12-17 18:12:34.552214+00:00,0.0,0.45,8956,32611,"{'coordinates': [[[-118.65239571447013, 36.294...",True,3.8,20181217_181234_1005,PS2,...,1005,1940776,155.0,26.1,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18 04:04:45.006000+00:00,0,0.1,24.298367,POLYGON ((-118.6523957144701 36.29426627663696...
2,2018-12-17 18:14:22.436537+00:00,0.0,0.92,8894,32611,"{'coordinates': [[[-118.64915467527891, 36.077...",False,3.8,20181217_181422_103e,PS2,...,103e,1940782,154.9,26.2,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18 04:26:30.431000+00:00,0,2.5,0.086941,POLYGON ((-118.6491546752789 36.07720201596842...
3,2018-12-17 18:14:21.402641+00:00,0.0,0.94,8874,32611,"{'coordinates': [[[-118.88762454600982, 36.264...",False,3.8,20181217_181421_103e,PS2,...,103e,1940782,154.9,26.1,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18 04:26:51.214000+00:00,0,2.5,29.270050,POLYGON ((-118.8876245460098 36.26414489594571...
4,2018-12-17 18:14:20.368745+00:00,0.0,0.90,8861,32611,"{'coordinates': [[[-118.86996154384987, 36.330...",True,3.8,20181217_181420_103e,PS2,...,103e,1940782,155.0,26.1,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-18 04:26:44.188000+00:00,0,2.5,33.831834,POLYGON ((-118.8699615438499 36.33029345943363...


In [17]:
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.1
good = scenes['quality_category']=="standard"
# recent = scenes["acquired"] > datetime.date(year=2017,month=1,day=1)
partial_coverage = scenes["overlap"] > 30
good_scenes = scenes[(good&clear&partial_coverage)]
display(good_scenes)
print(len(good_scenes))

# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.5
good = scenes['quality_category']=="standard"
all_time = scenes["acquired"] > datetime.date(year=2014,month=1,day=1)
full_coverage = scenes["overlap"] >= 60
all_scenes = scenes[(good&clear&all_time&full_coverage)]
display(all_scenes)
print(len(all_scenes))

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
8,2018-12-16 17:45:16.593668+00:00,0.00,0.00,8300,32611,"{'coordinates': [[[-118.85039421411327, 36.218...",True,3.6,20181216_174516_1052,PS2,...,1052,1938040,148.6,23.6,https://tiles.planet.com/data/v1/item-types/PS...,2018-12-17 04:14:18.252000+00:00,0,0.1,46.106849,"POLYGON ((-118.8503942141133 36.2186272036506,..."
81,2018-11-27 18:11:07.185794+00:00,0.01,0.05,8888,32611,"{'coordinates': [[[-118.67659523116637, 36.243...",True,3.9,20181127_181107_1038,PS2,...,1038,1882596,156.0,28.9,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-28 04:14:30.811000+00:00,0,2.9,50.540136,POLYGON ((-118.6765952311664 36.24337440123806...
82,2018-11-27 18:11:06.150850+00:00,0.00,0.05,8866,32611,"{'coordinates': [[[-118.6592223461988, 36.3083...",True,3.8,20181127_181106_1038,PS2,...,1038,1882596,156.1,28.9,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-28 04:13:30.548000+00:00,0,2.9,36.575564,POLYGON ((-118.6592223461988 36.30834731148848...
117,2018-11-20 17:49:25.237308+00:00,0.00,0.07,8357,32611,"{'coordinates': [[[-118.8622160017587, 36.1892...",True,3.7,20181120_174925_0f3b,PS2,...,0f3b,1864631,150.2,28.2,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-21 04:26:06.897000+00:00,0,0.1,37.343445,POLYGON ((-118.8622160017587 36.18926178233993...
181,2018-11-08 18:11:45.907064+00:00,0.00,0.09,8878,32611,"{'coordinates': [[[-118.49683434367596, 36.207...",True,3.8,20181108_181145_1035,PS2,...,1035,1829269,155.0,33.3,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-09 04:31:00.174000+00:00,0,0.1,42.126717,"POLYGON ((-118.496834343676 36.20739314517239,..."
186,2018-11-07 17:50:09.886822+00:00,0.00,0.04,8134,32611,"{'coordinates': [[[-118.43864078581348, 36.235...",True,3.6,20181107_175009_0f4a,PS2,...,0f4a,1827096,149.5,31.6,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-08 04:08:48.287000+00:00,0,1.6,54.927212,POLYGON ((-118.4386407858135 36.23524911757483...
187,2018-11-07 17:50:08.919511+00:00,0.00,0.05,8143,32611,"{'coordinates': [[[-118.4226013922241, 36.1740...",True,3.6,20181107_175008_0f4a,PS2,...,0f4a,1827096,149.5,31.7,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-08 04:09:29.003000+00:00,0,1.6,37.042972,"POLYGON ((-118.4226013922241 36.1740040675531,..."
189,2018-11-07 17:51:00.931453+00:00,0.00,0.04,8212,32611,"{'coordinates': [[[-118.52599176463528, 36.159...",True,3.6,20181107_175100_104d,PS2,...,104d,1826378,149.4,31.7,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-08 04:46:51.240000+00:00,0,0.2,43.463299,POLYGON ((-118.5259917646353 36.15999645889757...
196,2018-11-07 17:51:01.907678+00:00,0.00,0.02,8205,32611,"{'coordinates': [[[-118.54173843234429, 36.221...",True,3.6,20181107_175101_104d,PS2,...,104d,1826378,149.4,31.6,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-08 04:46:53.366000+00:00,0,0.2,54.838059,"POLYGON ((-118.5417384323443 36.2218685937915,..."
200,2018-11-06 18:12:24.166695+00:00,0.01,0.05,8949,32611,"{'coordinates': [[[-118.86494172922393, 36.257...",True,3.9,20181106_181224_1038,PS2,...,1038,1823572,154.8,33.9,https://tiles.planet.com/data/v1/item-types/PS...,2018-11-07 04:13:14.761000+00:00,0,0.1,34.682988,POLYGON ((-118.8649417292239 36.25743042386295...


22


/home/patrick/anaconda3/envs/sat_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  del sys.path[0]


TypeError: Cannot compare tz-naive and tz-aware datetime-like objects

In [18]:
# first create a list of colors
colors = ["#ff0000","#00ff00","#0000ff","#ffff00","#ff00ff","#00ffff"]
# grab our scenes from the geometry/footprint geojson
footprints = good_scenes["geometry"].tolist()
# for each footprint/color combo
for footprint,color in zip(footprints,colors):
    # create the leaflet object
    feat = {'geometry':footprint,"properties":{
            'style':{'color': color,'fillColor': color,'fillOpacity': 0.2,'weight': 1}},
            'type':u"Feature"}
    # convert to geojson
    gjson = GeoJSON(data=feat)
    # add it our map
    m.add_layer(gjson)
# now we will draw our original AOI on top 
feat = {'geometry':myAOI,"properties":{
            'style':{'color': "#FFFFFF",'fillColor': "#FFFFFF",'fillOpacity': 0.5,'weight': 1}},
            'type':u"Feature"}
gjson = GeoJSON(data=feat)
m.add_layer(gjson)   
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [21]:
imgs = []
# loop through our thumbnails and add display them
for img in good_scenes["thumbnail"].tolist():
    imgs.append(Image(url=f"{img}?api_key={PLANET_API_KEY}"))
    print (img)
display(*imgs)

https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181216_174516_1052/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181127_181107_1038/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181127_181106_1038/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181120_174925_0f3b/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181108_181145_1035/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181107_175009_0f4a/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181107_175008_0f4a/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181107_175100_104d/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181107_175101_104d/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181106_181224_1038/thumb
https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20181106_181223_1038/thumb
https://ti

In [23]:

def get_products(client, scene_id, asset_type='PSScene3Band'):    
    """
    Ask the client to return the available products for a 
    given scene and asset type. Returns a list of product 
    strings
    """
    out = client.get_assets_by_id(asset_type,scene_id)
    temp = out.get()
    return temp.keys()

def activate_product(client, scene_id, asset_type="PSScene3Band",product="analytic"):
    """
    Activate a product given a scene, an asset type, and a product.
    
    On success return the return value of the API call and an activation object
    """
    temp = client.get_assets_by_id(asset_type,scene_id)  
    products = temp.get()
    if( product in products.keys() ):
        return client.activate(products[product]),products[product]
    else:
        return None 

def download_and_save(client,product):
    """
    Given a client and a product activation object download the asset. 
    This will save the tiff file in the local directory and return its 
    file name. 
    """
    out = client.download(product)
    fp = out.get_body()
    fp.write()
    return fp.name

def scenes_are_active(scene_list):
    """
    Check if all of the resources in a given list of
    scene activation objects is read for downloading.
    """
    retVal = True
    for scene in scene_list:
        if scene["status"] != "active":
            print("{} is not ready.".format(scene))

            return False
    return True

In [73]:
to_get = good_scenes["id"].tolist()
activated = []
# for each scene to get
for scene in to_get:
    # get the product 
    product_types = get_products(client,scene)
    for p in product_types:
        # if there is a visual product
        if p == "visual": # p == "basic_analytic_dn"
            print("Activating {0} for scene {1}".format(p,scene))
            # activate the product
            _,product = activate_product(client,scene,product=p)
            activated.append(product)

Activating visual for scene 20181216_174516_1052
Activating visual for scene 20181127_181107_1038
Activating visual for scene 20181127_181106_1038
Activating visual for scene 20181120_174925_0f3b
Activating visual for scene 20181108_181145_1035
Activating visual for scene 20181107_175009_0f4a
Activating visual for scene 20181107_175008_0f4a
Activating visual for scene 20181107_175100_104d
Activating visual for scene 20181107_175101_104d
Activating visual for scene 20181106_181224_1038
Activating visual for scene 20181106_181223_1038
Activating visual for scene 20181105_175053_1050
Activating visual for scene 20181105_175052_1050
Activating visual for scene 20181102_175145_104a
Activating visual for scene 20181102_175144_104a
Activating visual for scene 20181031_175115_0f49
Activating visual for scene 20181026_180613_0e2f
Activating visual for scene 20181024_181155_0f4e
Activating visual for scene 20181023_181106_101f
Activating visual for scene 20181023_181105_101f
Activating visual fo

In [71]:
tiff_files = []
asset_type = "_3B_Visual"
# check if our scenes have been activated
if True: #scenes_are_active(activated):
    for to_download,name in zip(activated,to_get):
        # create the product name
        name = name + asset_type + ".tif"
        # if the product exists locally
        if( os.path.isfile(name) ):
            # do nothing 
            print("We have scene {0} already, skipping...".format(name))
            tiff_files.append(name)
        elif to_download["status"] == "active":
            # otherwise download the product
            print("Downloading {0}....".format(name))
            fname = download_and_save(client,to_download)
            tiff_files.append(fname)
            print("Download done.")
        else:
            print("Could not download, still activating")
else:
    print("Scenes aren't ready yet")

print(tiff_files)

Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
Could not download, still activating
[]


In [27]:

def load_image4(filename):
    """Return a 4D (r, g, b, nir) numpy array with the data in the specified TIFF filename."""
    path = os.path.abspath(os.path.join('./', filename))
    if os.path.exists(path):
        with rasterio.open(path) as src:
            b, g, r, nir = src.read()
            return np.dstack([r, g, b, nir])
        
def load_image3(filename):
    """Return a 3D (r, g, b) numpy array with the data in the specified TIFF filename."""
    path = os.path.abspath(os.path.join('./', filename))
    if os.path.exists(path):
        with rasterio.open(path) as src:
            b,g,r,mask = src.read()
            return np.dstack([b, g, r])
        
def get_mask(filename):
    """Return a 1D mask numpy array with the data in the specified TIFF filename."""
    path = os.path.abspath(os.path.join('./', filename))
    if os.path.exists(path):
        with rasterio.open(path) as src:
            b,g,r,mask = src.read()
            return np.dstack([mask])

def rgbir_to_rgb(img_4band):
    """Convert an RGBIR image to RGB"""
    return img_4band[:,:,:3]